In [18]:
from sklearn.datasets import load_digits
import numpy as np

In [2]:
n_class = 10
X, y = load_digits(n_class=n_class , return_X_y=True)

In [8]:
X.shape

(1797, 64)

## Ce dataset contient 1797 exemples et 64 caracteristiques. Il represente des images de chiffre de 0 à 9 la tache sous jaccente est une tache de classification d'images

# 2. Implémentez les fonctions permettant :

In [13]:
def onehot_encoding(y, n_class):
    n = y.shape[0]
    y_one = np.zeros((n, n_class))
    for i in range(n):
       y_one[i, y[i]] = 1
    return y_one

In [22]:
def softmax(z):
    z -= np.max(z) # computational trick for numerical stability
    som=np.sum(np.exp(z))
    sm=np.array([np.exp(i)/som for i in z])
    return sm

In [23]:
def get_prob_pred(X, W):
    z=X.T@W
    probs=np.max(softmax(z))
    preds=np.argmax(softmax(z))
    return probs , preds